In [220]:
#Importing the required packages
import os
import re
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import OrderedDict


In [187]:
#Changing the working directory
os.chdir('E:/Capstone IATH/Code')

#Setting the path
path = 'E:/Capstone IATH/Data/books'

#Getting all the files and removing the ones that are not the necessary
files = []
for r, d, f in os.walk(path):
    for file in f:
        if ".bess" in file:
            files.append(os.path.join(r, file))
            
files = [f.replace('\\', '/') for f in files]
files = files[1:343]
index = files.index(str(path) + '/a221/a221.bio05.bess.xml')
del files[index]

In [188]:
#Setting up the urls
link = [str(f.split('/')[-3])+ '/' + str(f.split('/')[-2]) for f in files]
urls = ['http://cbw.iath.virginia.edu/exist/cbw/dual/' + str(l) for l in link]

In [189]:
#Creating an empty dataframe to store the data
data = pd.DataFrame()

In [190]:
for url in urls:
    html = requests.get(url)
    soup = BeautifulSoup(html.text,'lxml')
    pno = soup.find_all(class_ = 'PNum')
    pt = soup.find_all(class_ = 'text')
    parano = [p.text for p in pno]
    text = [t.text for t in pt]
    col = [('ParagraphNo', parano), ('ParagraphText', text)]
    df = pd.DataFrame.from_dict(OrderedDict(col))
    df['CollectionID'] = url.split('/')[6]
    df['BiographyID'] = url.split('/')[7]
    data = data.append(df)

In [232]:
#Cleaning the text column to remove duplicate spaces, newline characters tab characters, leading and trailing spaces
text = data['ParagraphText']
text = [t.replace("\n","").strip() for t in text]
text = [t.replace("\t","") for t in text]
text = [re.sub(' +', ' ', t) for t in text]
data['ParagraphText'] = text

In [233]:
#Writing to a csv file
data.reset_index(drop=True, inplace=True)
data.to_csv("textdata.csv", index = False)